In [1]:
print("hhi")

hhi


In [2]:
from dotenv import load_dotenv
import os

load_dotenv()


True

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
google_gemini = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=0.9)
google_gemini.invoke("What is the capital of France?").content


[{'type': 'text',
  'text': 'The capital of France is **Paris**.',
  'extras': {'signature': 'EsIBCr8BAb4+9vtDVBFZ+4m15qSwSsX2QeWk4CNpEZ6wKe45z+aRYKREWobGlzEcd0EZ+zhLKbdqs1ThVbkqaYSUhh9Jx8NcgYsLozDAbw/3yqpmcOOzhV7bBe+aDLlRlP/nwjXKXQKwEVSL4+BQNeSawi9fqASn1jnFgnfq5rtfwhPobKkzgyrg2Sl9atY65CBsae5ocFV3/YDK4yofwn/pLAV0m+/jJ1T3lEbXlKhsfxBbLoF0G+5f7EHxwVIC0CuMKhQ='}}]

In [4]:
from typing_extensions import Annotated,TypedDict
import operator

In [5]:
from langchain_core.messages import ChatMessage, HumanMessage, SystemMessage, AIMessage,AnyMessage

In [6]:
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    

In [7]:
def llm_call(state: GraphState) -> dict:
    """Call the LLM with messages and append AI response."""
    response = google_gemini.invoke(state["messages"])
    return {
        "messages": [AIMessage(content=response.content)]
    }


def token_counter(state: GraphState) -> dict:
    """Count tokens (simple word count) in the last AI message."""
    last_msg = state["messages"][-1]
    text = last_msg.content
    if isinstance(text, list):
        text = " ".join(str(item) for item in text)
    token_number = len(text.split())
    summary = f"Total token number in the generated answer (word count) is {token_number}"
    return {
        "messages": [AIMessage(content=summary)]
    }

In [8]:
from langgraph.graph import StateGraph


In [9]:
builder=StateGraph(GraphState)


In [10]:
builder.add_node(llm_call)
builder.add_node(token_counter)


In [11]:
builder.set_entry_point("llm_call")
builder.add_edge("llm_call", "token_counter")

builder.set_finish_point("token_counter")

In [12]:
app=builder.compile()

In [13]:
result = app.invoke({"messages": [HumanMessage(content="Hi, this is rakesh. Say hello in detail.")]})


In [14]:
result 

{'messages': [HumanMessage(content='Hi, this is rakesh. Say hello in detail.', additional_kwargs={}, response_metadata={}),
  AIMessage(content=[{'type': 'text', 'text': 'Hello there, Rakesh! It is a pleasure to meet you.\n\nSince you asked for a detailed greeting, I want to take a moment to properly welcome you. Whether you are here to brainstorm new ideas, solve a complex problem, learn something new, or simply have a friendly conversation, I am delighted to be at your service.\n\nThe name **Rakesh** is a beautiful one, traditionally originating from Sanskrit, often translated as "Lord of the Full Moon" (*Raka* meaning full moon and *isha* meaning lord). It’s a name that carries a sense of light and calm, and I hope our interaction today reflects that same positive energy.\n\nAs an AI, I am equipped to help you with a wide variety of things, such as:\n*   **Creative Writing:** Crafting stories, poems, or professional emails.\n*   **Learning:** Explaining complex topics in science, hi

In [15]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [16]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=500)

In [19]:
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)


In [21]:
wiki_tool.run({"query":"Paris capital France"})

"Page: Paris\nSummary: Paris is the capital and largest city of France, with an estimated city population of 2,047,602 in an area of 105.4 km2 (40.7 sq mi), and a metropolitan population of 13,239,090 as of January 2026. Located on the river Seine in the centre of the Île-de-France region, it is the largest metropolitan area and fourth-most populous city in the European Union (EU). Nicknamed the City of Light, partly because of its role in the Age of Enlightenment, Paris has been one of the world'"

In [27]:
from dotenv import load_dotenv
import os
load_dotenv()
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")


In [28]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool=TavilySearchResults(tavily_api_key=TAVILY_API_KEY)

In [30]:
tool.invoke("score the scorecard of ind vs  nam in worldcup 2026")

[{'title': 'India vs Namibia Live Score T20 World Cup 2026 - Firstpost',
  'url': 'https://www.firstpost.com/firstcricket/india-vs-namibia-live-score-t20-world-cup-2026-ind-vs-nam-scorecard-updates-delhi-liveblog-13979042.html',
  'content': 'Firstpost\n\n### Sections\n\n### Shows\n\n### Events\n\nfp-logo\n\n### Sections\n\n### Shows\n\n### Events\n\n# India vs Namibia Live Score T20 World Cup 2026: India secure convincing 93-run victory\n\n## IND vs NAM LIVE Score (India national cricket team vs Namibia national cricket team scorecard): India have defeated Namibia by 93 runs thanks to a solid bowling performance. India scored 209/9 before bowling Namibia out for just 116.\n\nIndia vs Namibia Live Score T20 World Cup 2026: India secure convincing 93-run victory "India vs Namibia Live Score T20 World Cup 2026: India secure convincing 93-run victory") [...] India vs Namibia Live Score, T20 World Cup: India have struck five times already as they look set for another comfortable victory ag

In [31]:
from langchain_community.tools import DuckDuckGoSearchRun
search=DuckDuckGoSearchRun()



In [32]:
search.invoke("the heeight of himalaya mountain")

"Overall, the Himalayan mountain system is the world's highest , and is home to 10 of 14 of the world's highest peaks, the Eight -thousanders, and more than 50 peaks over 7,000 metres (23,000 ft). The Karakoram and Hindu Kush are regarded as separate ranges whose peaks are not included in this table. See full list on en.wikipedia.org Feb 2, 2026 · Himalayas are a great mountain system of Asia forming a barrier between the Plateau of Tibet to the north and the alluvial plains of the Indian subcontinent to the south. The Himalayas include the highest peaks in the world, most notably Mount Everest. Learn more about the mountain system. View all By contrast, the highest peak outside Asia (Aconcagua, in the Andes) is 6,961 meters (22,838 feet) tall . The Himalaya -Karakoram range contains 96 of the world's 109 peaks over 24,000 feet. The Himalayan Mountains are the highest and one of the youngest mountain ranges in the world, stretching about 2,400 kilometers (1,500 miles) across South Asia

In [34]:
from langchain.tools import tool


In [35]:
@tool
def add(a:int,b:int):
    ''' this is a addition tool'''
    return a+b

In [37]:
add.invoke({"a": 5, "b": 3})

8